# JMS to Reactive example with HelloWorld-MDB and CoolStuff Store
## Overview
We will use 2 sample applications for this example.  Both of them have an original branch which represents their starting point as a Java EE application and then another branch for their end result of being migrated to Quarkus.  
## What are we doing?
* We will run a source code analysis on both sample applications original state as a Java EE app
* We will attempt to modernize the 'Cool Stuff Store' JavaEE branch to Quarkus by:
    * Use the static code analysis as guides of what changes need to be made to what files
    * Combine the static code analysis incidents with a sample of how these issues were solved in the 'Hello World MDB' Quarkus branch.
        * The intent is to use what was done in 'Hello World MDB' Quarkus branch to heavily influence how the LLM solves a similar issue with 'Cool Stuff Store'
## Sample App Information
### Hello World MDB
* This example started from JBoss EAP Quickstarts: https://github.com/jboss-developer/jboss-eap-quickstarts/tree/7.4.x/helloworld-mdb/src/main/java/org/jboss/as/quickstarts/mdb
* Our team then ported 'HelloWorld MDB' to Quarkus:https://github.com/savitharaghunathan/helloworld-mdb/tree/quarkus
    * Note these blog posts from prior work of modernizing this same HelloWorld app, the difference with our latest work and the below is the change our team did to also address JMS to Reactive changes:
        * https://developers.redhat.com/blog/2019/11/07/quarkus-modernize-helloworld-jboss-eap-quickstart-part-1#let_s_modernize_helloworld
        * https://developers.redhat.com/blog/2019/11/08/quarkus-modernize-helloworld-jboss-eap-quickstart-part-2
### Cool Stuff Store
* https://github.com/deewhyweb/eap-coolstore-monolith/tree/ocp
    * Jave EE 7 application
* https://github.com/mathianasj/eap-coolstore-monolith/tree/quarkus-migration
    * Note we have 2 branches to work with, this will be the repo we use in our example
        * Java EE:  https://github.com/mathianasj/eap-coolstore-monolith/tree/main
        * Quarkus: https://github.com/mathianasj/eap-coolstore-monolith/tree/quarkus-migration
* Related blog posts using this app for other purposes:
    * https://developers.redhat.com/articles/2023/08/23/how-migrate-complex-jboss-eap-application-openshift#

* We developed rules for JMS -> Reactive here:
    * https://github.com/windup/windup-rulesets/pull/1043
    * https://github.com/jmle/rulesets/blob/jms-rule/default/generated/quarkus/05-jms-to-reactive-quarkus.windup.yaml



In [1]:
# Bring in the common deps
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.callbacks import FileCallbackHandler

from loguru import logger
import os
import pprint

pp = pprint.PrettyPrinter(indent=2)
output_dir = "output/helloworldmdb"
try:
     os.makedirs(output_dir, exist_ok=True)
except OSError as error:
    print(f"Error creating directory {output_dir}: {error}")
    raise error

logfile = os.path.join(output_dir, "output_06_jms_helloworld.log")
with open(logfile, "a") as f:
    # We want to start each run with a clean log file
    f.truncate(0)

def get_diff_in_result(text: str):
    _, after = text.split("```diff")
    return after.split("```")[0]




# Naive Zero Shot Example
* We want to see what kind of response do we get back with a very simple zero-shot prompt

In [2]:
from kyma_poc.scm import GitDiff

example_git_path = "../data/helloworld-mdb"
gd = GitDiff(example_git_path)

# Pick a file to update
file_path = "src/main/java/org/jboss/as/quickstarts/mdb/HelloWorldQueueMDB.java"

# Get the file contents  
# JavaEE code is in 'main' branch
# Quarkus code is in 'quarkus' branch
example_file_contents = gd.get_file_contents(file_path, gd.get_commit_from_branch("main"))
#print(example_file_contents)

template = """
    You are an AI Assistant trained on migrating enterprise JavaEE code to Quarkus.
    I will give you an example of a JavaEE file and you will give me the Quarkus equivalent.
    Approach this code migration from Java EE to Quarkus as if you were an experienced developer.
    Prepare a reasoning and summary of each step that is required and why.
    After you have shared your step by step thinking, provide a full output of the updated file:

    ## Original input file in Java EE to update to Quarkus
    Filename: "{file_path}"

    ### Source code
    ```java 
    {example_file_contents}
    ```

    Structure your ouput in Markdown format such as:

    ## Reasoning
    Write the step by step reasoning in this markdown section.
    If you are unsure of a step or reasoning, clearly state you are unsure and why.

    ## Updated File
    ```java
        Write the updated file for Quarkus in this section
    ```
   
    """

template_args = {
    "file_path": file_path,
    "example_file_contents": example_file_contents,
}
formatted_prompt = template.format(**template_args)
saved_formatted_template_path = os.path.join(output_dir, "simple_formatted_template.md")
with open(saved_formatted_template_path, "w") as f:
    f.truncate()
    f.write(formatted_prompt)

# Ask the LLM to update
llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-16k")
prompt = PromptTemplate.from_template(template)
chain = LLMChain(llm=llm, prompt=prompt)

result = chain.run(template_args)
# Save the output
simple_result_path = os.path.join(output_dir, "simple_result.md")
with open(simple_result_path, "w") as f:
    f.truncate()
    f.write(result)

# Extract the updated java code and save it
def get_java_in_result(text: str):
    _, after = text.split("```java")
    return after.split("```")[0]

updated_file_contents = get_java_in_result(result)
updated_file_path = os.path.join(output_dir, "simple_updated_file.java")
with open(updated_file_path, "w") as f:
    f.truncate()
    f.write(updated_file_contents)

# Save the original source code as well for comparison
original_file_path = os.path.join(output_dir, "simple_original_file.java")
with open(original_file_path, "w") as f:
    f.truncate()
    f.write(example_file_contents)


# Compare the LLM result to what we have in the Quarkus branch 


# Zero Shot with Static Code Analysis Info
* We will pick a single file and form a template that includes information from our static code analysis to ask the LLM to help update towards Quarkus


In [3]:
from kyma_poc import Report

def display_impacted_file(impacted_file: dict):
    print(f"File: {impacted_file['file_path']}")
    for i in impacted_file['incidents']:
        print(f"  Ruleset Description: {i['ruleset_description']}")
        print(f"  Violation: {i['violation_name']}")
        print(f"  Violation Description: {i['violation_description']}")
        print(f"  Message: {i['message']}")

def create_zero_shot_template_from_incidents(source_code, incidents: list):
    for i in incidents:
        template += f"Rule: {i['ruleset_description']}\n"
        template += f"Violation: {i['violation_name']}\n"
        template += f"Message: {i['message']}\n"
        template += f"```diff\n"
        template += f"{i['violation_description']}\n"
        template += f"```\n"
    return template

hello_world_mdb_jms_yaml = "../data/example_reports/helloworld-mdb_custom_rules/output.yaml"

rObj = Report(hello_world_mdb_jms_yaml)
impacted_files = rObj.get_impacted_files()
for f in impacted_files.keys():
    display_impacted_file(impacted_files[f])



SyntaxError: unterminated string literal (detected at line 12) (2300332131.py, line 12)

In [ ]:

template = """You are an excellent enterprise architect who has an extensive
background in helping companies rewrite their legacy Java EE applications to Quarkus.

You will read a user's problem along with examples of how they have solved a problem in the past.
The past examples will be presented in format of a summary of the issue along with source code of 
that point in time along with the updated source code when the problem is fixed

You will then write Quarkus code to solve their current problem.
You will output the results in the form a diff which can be applied via 'git apply'.

Example #1 Issue: {example1_issue}

Example #1 Original Source Code:
```java
{example1_original_code}
```

Example #1 Solved Source Code:
```java
{example1_solved_code}
```

Current Issue: 
{current_issue}

Current Issue Original Source Code: 
```java
{current_issue_original_code}
```

Your job is to look at the 'Current Issue' and the 'Current Issue Original Source Code' 
and rewrite the 'Current Issue Original Source Code' so the 'Current Issue' is solved 
in a manner similar to how 'Example #1 Original Source Code' was rewritten to 
'Example #1 Solved Source Code' 

Think through the changes you will make and explain each step of the process.
If you are unsure of what changes is needed please state you are unsure and ask 
for clarification to help you.

When you are done explaining the reasoning for each change, write the updated 
Quarkus source code in the form of a diff which can be applied via 'git apply' 
in Markdown format, e.g.:

```diff
....
```"""

In [ ]:
# First example is to assess what kind of response do we get back from the LLM if we do a very simple zero-shot example
